[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/ipml/blob/master/tutorial_notebooks/8_data_prep_solutions.ipynb) 

# Data preparation


<hr>
<br>

This is the second notebook in a series revisiting our lecture on EDA and data preparation. Having elaborated on the `Pandas` library and having illustrated the use of standard plotting libraries in [Tutorial 7](), we turn our attention to the data preparation part of the lecture. In this scope, we investigate options to scale and categorize continuous features and properly encode categorical features before using them in a machine learning model. We will see that most of the needed functionality is readily available in `Pandas`.  

We continue to use our credit risk data set for this tutorial. Before moving on, we perform some standard setup tasks, which, by now, you know well. 


In [ ]:
# Load standard libraries
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load credit risk data directly from GitHub
data_url = 'https://raw.githubusercontent.com/Humboldt-WI/bads/master/data/hmeq.csv'
hmeq = pd.read_csv(data_url)

# Preview the data
hmeq.info()

# Altering Data Types
We start with a rather technical bit, data types. Recall the way our data is stored at the moment. 
The features *JOB* and *REASON* are stored as data type `object`. This is the most general data type in Python. A variable of this type can store pretty much any piece of data, numbers, text, dates, times, ... This generality has a price. First, storing data as data type `object` consumes a lot of memory. Second, we cannot access specific functionality that is available for a specific data type only. Functions to manipulate text are an example. These are available for data of type `string` but not for data of type `object`. 
<br>
In our case, the two features that Pandas stores as objects are actually categorical variables. We can easily verify this using, e.g., the `value_counts` method.

In [ ]:
print(hmeq.REASON.value_counts())  # so REASON is a binary variable

In [ ]:
print(hmeq.JOB.value_counts())  # JOB is a categorical variable with many levels

Knowing our two "object features" are categories, we should alter their data type accordingly. To that end, we make use of the function `astype`, which facilitates converting one data type into another. 

In [ ]:
# Code categories properly 
hmeq['REASON'] = hmeq['REASON'].astype('category')
hmeq['JOB'] = hmeq['JOB'].astype('category')
hmeq.info()  # verify the conversion was successful

Although it does not really matter for this tiny data set, note that the conversion from object to category has reduced the amount of memory that the data frame consumes. On my machine, we need 524.2 KB after the translation, whereas we needed more than 600 KB for the original data frame. If you work with millions of observations the above conversion can result in a significant reduction of memory consumption. If memory consumption is an issue, we can achieve a significant further reduction by reducing the precision of the numerical variables. *Downcasting* from float64 to float32 is likely ok for predictive modeling. Also, the target variable is stored as an integer but we know that it has only two states. Thus, we can convert the target to a boolean. We perform these transformations in the next code demo.

In [5]:
# The target variable has only two states so that we can store it as a boolean
hmeq['BAD'] = hmeq['BAD'].astype('bool')

# For simplicity, we also convert LOAN to a float so that all numeric variables are of type float
hmeq['LOAN'] = hmeq['LOAN'].astype(np.float64)

# Last, let's change all numeric variables from float64 to float32 to reduce memory consumption
num_vars = hmeq.select_dtypes(include=np.float64).columns
hmeq[num_vars] = hmeq[num_vars].astype(np.float32)

Invest some time to understand the above codes. Our examples start to combine multiple pieces of functionality. For example, the above demo uses indexing, functions, and function arguments to perform tasks. Keep practicing and you will become familiar with the syntax.
<br>
Finally, let's verify our changes once more.

In [ ]:
# Check memory consumption after the conversions
hmeq.info()

In total, our type conversions reduced memory consumption by more than a half. You might want to bear this potential in mind when using your computer to process larger data sets. Should you be interested in some more information on memory efficiency, have a look at this post at [TowardDataScience.com](https://towardsdatascience.com/pandas-save-memory-with-these-simple-tricks-943841f8c32). 

# Missing values
Our data contains many missing values. This is easily seen when calling, e.g., `hmeq.head(5)`, and is common when working with real data. Likewise, handling missing values is a standard task in data preparation. `Pandas` provides the function `.isna()` as entry point to the corresponding functionality and helps with identifying the relevant cases.

*Note*: `Pandas` also supports an equivalent function called `.isnull()`. 

In [ ]:
# Boolean mask of same size as the data frame to access missing values via indexing
missing_mask = hmeq.isna()

print(f'Dimension of the mask: {missing_mask.shape}')
print(f'Dimension of the data frame: {hmeq.shape}')

missing_mask


We can now count the number of missing values per row or per column or in total.

In [ ]:
# missing values per row
miss_per_row = missing_mask.sum(axis=1)
print('Missing values per row:\n', miss_per_row)
print('*' * 50)
# missing values per column
miss_per_col = missing_mask.sum(axis=0)
print('Missing values per column:\n', miss_per_col )
print('*' * 50)
# count the total number of missing values
n_total_missing = missing_mask.sum().sum()
print(f'Total number of missing values: {n_total_missing}')
print('*' * 50)

It can be useful to visualize the *missingness* in a data set by means of a heatmap. Note how the below example gives you a good intuition of how and where the data set is affected by missing values. 

In [ ]:
sns.heatmap(hmeq.isna())  # quick visualization of the missing values in our data set
plt.show()

## Categorical features
Let's start with the two categorical features, `REASON` and `JOB`. We will treat them differently for the sake of illustration. Now that we start altering our data frame more seriously, it is a good idea to make a copy of the data so that we can easily go back to a previous state.

In [10]:
# copy data: we continue with altering the variable df while we keep variable hmeq for the raw data
df = hmeq.copy()

### Adding a new category level
One way to treat missing values in a categorical feature is to introduce a new category level "IsMissing". We will demonstrate this approach for the feature *REASON*. 
<br>One feature of the category data type in Pandas is that category levels are managed. We cannot add levels directly. Thus, before assigning the missing values our new category level *IsMissing*, we first need to introduce this level. We basically tell our data frame that *IsMissing* is another suitable entry for *REASON* next to the levels that already exist in the data frame. 

In [ ]:
# Variable REASON: we treat missing values as a new category level.
# First we need to add a new level
df.REASON = df.REASON.cat.add_categories(['IsMissing'])

# Now we can do the replacement
df.REASON[df.REASON.isna() ] = "IsMissing"
df.REASON.head()

In [ ]:
df.REASON.isna().sum()  # verify that no more missing values exist

### Mode replacement
For the feature *JOB*, which is multinomial, we replace missing values with the mode of the feature. Please note that this is a crude way to handle missing values. I'm not endorsing it! But you should have at least seen a demo. Here it is. 

In [ ]:
# Determine the mode
mode_of_job = df.JOB.mode()
print(mode_of_job)

In [ ]:
# replace missing values with the mode
df.JOB[df.JOB.isna() ] = df.JOB.mode()[0]  # the index [0] is necessary as the result of calling mode() is a Pandas Series
# verify that no more missing values exist
df.JOB.head()

In [ ]:
# Verify more seriously that missing value replacement was successful
if df.REASON.isna().any() == False and df.JOB.isna().any() == False:
    print('well done!')
else:
    print('ups')

## Numerical features
We have a lot of numerical features. To keep things simple, we simply replace all missing values with the median. Again, this is  a crude approach that should be applied with care; if at all. However, it nicely shows how we can process several columns at once using a loop. Further, note the use of the method `fillna()` to replace missing values. We could have also used it in our previous examples concerning categorical features. It is the recommend way to replace missing values in `Pandas`.

In [16]:
for col in df.select_dtypes(include='float32').columns:  # loop over all numeric columns
    if df[col].isna().sum() > 0:                         # check if there are any missing values in the current feature
        m = df[col].median(skipna=True)                  # compute the median of that feature
        df[col].fillna(m, inplace=True)                  # replace missing values with the median

Should you wonder whether it is necessary to write a loop to perform this rather standard operation, the answer is no. You could achieve the same result more elegantly when combining the `fillna()` method with a call to the method `transform()`. Here is how this would look like:
```python
# Alternative approach to impute missing values with the feature median
cols = df.select_dtypes(include='float32').columns 

df[cols] = df[cols].transform(lambda x: x.fillna(x.median()))
``` 
The function `transform()` applies a function to each column of the DataFrame. The lambda function takes each column, fills the missing values with the median of that column, and returns the transformed column. This way, you avoid looping over each column manually. This version can be considered more elegant, but our first shot, writing a loop, may legitimately be considered more readable.

In [ ]:
# Verify there are no more missing values in the data
n_total_missing = df.isna().sum().sum()
if  n_total_missing == 0:
    print('Well done, no more missing values!')
else:
    print(f'Ups! There are still {n_total_missing} missing values.')


# Exercises

## 1. Outliers
The lecture introduced a rule of thumb saying that, for a given feature, a feature value $x$ can be considered an outlier if 
$$x >q_3(X) + 1.5 \cdot IQR(X)$$

where $q_3(X)$ denotes the third quantile of the distribution of feature $X$ and $IQR(X)$ the corresponding inter-quartile range.

1. Use the `Pandas` method `quantile` to compute the third and first quartile of feature `LOAN`.
2. Compute the threshold value that a feature value $x$ must not exceed according to the above equation. Store the result in a variable. 
3. Use logical indexing to identify all upper outliers in the feature `LOAN`.
4. Create a new data frame that has no outliers in the feature `LOAN`. To that end: 
- Reuse your solution to task 3 to identify outliers using indexing
- Change the `LOAN` values for all outlier cases to the threshold you computed in step 2.

Follow-up, more advanced tasks:<br>

5. Write a custom function that implements the functionality you created in task 4. Make the feature to work on an argument of your function.
6. Call your custom function for all numerical features in the data frame. The goal is to create a data frame that does not have any upper outlier in any of its numerical features. To demonstrate the capabilities of your function, set the threshold to $3 \cdot IQR(X)$. This way, only extreme outliers will be removed.

In [ ]:
# 1. First and third quantile of the LOAN feature
quantiles = df["LOAN"].quantile(q=[0.25, 0.75])

# To extract the actual numbers into easy-to-use variables,
# we can first create a tuple and then use unpacking
q1, q3 = (quantiles.values)
print(f"The first and third quartile are, respectively {q1} and {q3}")

In [ ]:
# 2. Threshold value for upper outlier
tau = q3 + 1.5*(q3-q1)
tau

In [ ]:
# 3. Find upper outliers in the LOAN feature
ix_upper_outlier = df["LOAN"]>tau 
df.loc[ix_upper_outlier, "LOAN"]

In [ ]:
# 4. Remove upper outliers in feature LOAN
df.loc[ix_upper_outlier, "LOAN"] = tau  # outlier truncation
df.loc[ix_upper_outlier, "LOAN"]  # print results to see the effect

In [22]:
# 5. Customer function for outlier detection and removal 
def outlier_truncation(x, factor=1.5):
    """
    Identifies outlier values based on the inter-quartile range IQR. 
    Corresponding outliers are truncated and set to a contant value equal to the IQR
    times a factor, which, following Tuckey's rule, we set to 1.5 by default
    
        Parameters:
            x (Pandas Series): A data frame column to scan for outliers
            factor (float): An outlier is a value this many times the IQR above q3/below q1
            
        Returns:
            Adjusted variable in which outliers are truncated
    """
    x_new = x.copy()
    
    # Calculate IQR
    IQR = x.quantile(0.75) - x.quantile(0.25) 
    
    # Define upper/lower bound
    upper = x.quantile(0.75) + factor*IQR
    lower = x.quantile(0.25) - factor*IQR
    
    # Truncation
    x_new[x < lower] = lower.astype(np.float32)  # downcasting to float32 is needed to ensure
    x_new[x > upper] = upper.astype(np.float32)  # compatibility with how we store the data in our data frame 
    
    return x_new


In [ ]:

# 6. Application of the function to all numerical features in the data 

# Select numeric variables for outlier treatment. 
ix_numerical = df.select_dtypes(include="float32").columns

# Process every selected column using apply
# Updated 10.06.20 to show passing arguments to the 'applied' functions. Just send a tuple with arguments in the order as specified
# by the called function leaving out the first argument (see, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html)
df[ix_numerical] = df[ix_numerical].apply(outlier_truncation, axis=0, factor=3)  
df.describe()

## 2. Scaling numerical features
Another common data preparation task is scaling numerical features. The goal is to ensure that all features have the same scale. This is important for many machine learning algorithms. The lecture introduced two common scaling methods: min-max scaling and z-score scaling.
The `sklearn` library provides implementations of both approaches in the classes `MinMaxScaler` and `StandardScaler`, which are part of the module `preprocessing`. Experiment with these classes to solve the following exercises.

1. Import the class `MinMaxScaler` and `StandardScaler` from the module `preprocessing` in the library `sklearn`.
2. Familiarize yourself with the functioning of the `StandardScaler` using its documentation and other sources (e.g., web search). 
3. Test the `StandardScaler` by applying it to the numerical features `LOAN`. Afterwards, the scaled feature should have a mean of 0 and a standard deviation of 1. Write a few lines of code to verify this.
4. The use of the `MinMaxScaler` is similar to the `StandardScaler`. Apply the `MinMaxScaler` to all other numerical features in the data set. More specifically, 
- Create a new data frame that contains only the numerical features.
- Remove the feature `LOAN` from that data frame; as we already scaled it in task 3.
- Apply the `MinMaxScaler` to the new data frame.
- Write a few lines of code to verify that the scaling was successful. To that end, recall what the 'MinMaxScaler' does.
- Combine the scaled features with the feature `LOAN` and the categorical features in a new `DataFrame`.


In [ ]:
# 1. Import the class MinMaxScaler and StandardScaler from the module preprocessing in the library sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 2. Familiarize yourself with the functioning of the StandardScaler using its documentation and other sources (e.g., web search).
# For example, you could start here: https://scikit-learn.org/1.5/api/sklearn.preprocessing.html

# 3. Test the StandardScaler by applying it to the numerical feature LOAN. Afterwards, the scaled feature should have a mean of 0 and a standard deviation of 1. Write a few lines of code to verify this.
scaler = StandardScaler()
df["LOAN_scaled"] = scaler.fit_transform(df[["LOAN"]])

# Verify the scaling
print(f"Mean of LOAN_scaled: {df['LOAN_scaled'].mean()}")
print(f"Standard deviation of LOAN_scaled: {df['LOAN_scaled'].std()}")


In [ ]:
# 4. Apply the MinMaxScaler to all other numerical features in the data set.
# Create a new data frame that contains only the numerical features.
df_numerical = df.select_dtypes(include="float32").copy()

# Remove the feature LOAN from that data frame; as we already scaled it in task 3.
df_numerical.drop(columns=["LOAN"], inplace=True)

# Apply the MinMaxScaler to the new data frame.
min_max_scaler = MinMaxScaler()
df_numerical_scaled = pd.DataFrame(min_max_scaler.fit_transform(df_numerical), columns=df_numerical.columns)

# Verify the scaling
print(f"Min values of scaled features:\n{df_numerical_scaled.min()}")
print(f"Max values of scaled features:\n{df_numerical_scaled.max()}")

# Combine the scaled features with the feature LOAN and the categorical features in a new DataFrame.
df_scaled = pd.concat([df[["LOAN_scaled"]], df_numerical_scaled, df.select_dtypes(include=["category", "bool"])], axis=1)
df_scaled.head()

## 3. Discretizing numerical features
Discretizing numerical features is another common data preparation task. The goal is to convert continuous numerical features into discrete bins or categories. This can be useful for certain types of analysis and modeling. The `pandas` library provides the `cut` and `qcut` functions for this purpose.

1. Familiarize yourself with the `cut` and `qcut` functions in the `pandas` library using their documentation and other sources (e.g., web search).
2. Use the `cut` function to discretize the `LOAN` feature into 5 equal-width bins. Assign meaningful labels to each bin (e.g., 'Very Low', 'Low', 'Medium', 'High', 'Very High').
3. Verify the binning by displaying the first few rows of the data frame and checking the `LOAN` feature.
4. Use the `qcut` function to discretize the `MORTDUE` feature into 4 quantile-based bins. Assign meaningful labels to each bin (e.g., 'Q1', 'Q2', 'Q3', 'Q4').
5. Verify the binning by displaying the first few rows of the data frame and checking the `MORTDUE` feature.

Follow-up, more advanced tasks:<br>

6. Create a new data frame that includes the discretized `LOAN` and `MORTDUE` features along with the other original features.
7. Write a custom function that takes a data frame and a list of numerical features as input and returns a new data frame with all specified features discretized into a given number of bins using the `cut` function. Test your function on the numerical features in the data frame.

In [ ]:
# 2. Use the cut function to discretize the LOAN feature into 5 equal-width bins. Assign meaningful labels to each bin (e.g., 'Very Low', 'Low', 'Medium', 'High', 'Very High').
loan_bins = pd.cut(df["LOAN"], bins=5, labels=["Very Low", "Low", "Medium", "High", "Very High"])
df["LOAN_bins"] = loan_bins  # We add a new column as opposed to overwriting the existing column

# 3. Verify the binning by displaying the first few rows of the data frame and checking the LOAN feature.
print(df[["LOAN", "LOAN_bins"]].head(10))

In [ ]:
# 4. Use the qcut function to discretize the MORTDUE feature into 4 quantile-based bins. Assign meaningful labels to each bin (e.g., "Q1", "Q2", "Q3", "Q4").
mortdue_bins = pd.qcut(df["MORTDUE"], q=4, labels=["Q1", "Q2", "Q3", "Q4"])

# 5. Verify the binning by displaying the first few rows of the data frame and checking the MORTDUE feature.
df["MORTDUE_bins"] = mortdue_bins
print(df[["MORTDUE", "MORTDUE_bins"]].head(10))

In [ ]:
# 6. Create a new data frame that includes the discretized LOAN and MORTDUE features along with the other original features.
df_discretized = df.drop(columns=["LOAN", "MORTDUE"])
df_discretized = pd.concat([df_discretized, loan_bins, mortdue_bins], axis=1)
df_discretized  # preview the data

In [ ]:

# 7. Write a custom function that takes a data frame and a list of numerical features as input and returns a new data frame with all specified features discretized into a given number of bins using the cut function. Test your function on the numerical features in the data frame.
def discretize_features(df, features, bins=5, labels=None):
    """
    Discretizes the specified columns of a DataFrame into equal-width bins.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data to be discretized.
    columns (list of str): The list of column names to be discretized.
    bins (int): The number of equal-width bins to use for discretization.

    Returns:
    pandas.DataFrame: A new DataFrame with the specified columns discretized into bins.

    """
    df_discretized = df.copy()
    for feature in features:
        df_discretized[feature + "_bins"] = pd.cut(df_discretized[feature], bins=bins, labels=labels)
    return df_discretized

# Test the function on the numerical features in the data frame
ix_numerical = df.select_dtypes(include="float32").columns
df_discretized_all = discretize_features(df, ix_numerical, bins=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
df_discretized_all.head()

## Encoding categorical features
Encoding categorical features is another important data preparation task. Machine learning algorithms require numerical input, so we need to convert categorical features into numerical format. The `pandas` library provides the `get_dummies` function for this purpose. This function will turn a categorical feature with k levels into k binary features, as shown in this example for a categorical feature COLOR:

**Original Table**

| ID | COLOR  |
|----|--------|
| 1  | Red    |
| 2  | Blue   |
| 3  | Green  |
| 4  | Red    |
| 5  | Green  |

**Table After Dummy Coding**

| ID | COLOR_Red | COLOR_Blue | COLOR_Green |
|----|-----------|------------|-------------|
| 1  | 1         | 0          | 0           |
| 2  | 0         | 1          | 0           |
| 3  | 0         | 0          | 1           |
| 4  | 1         | 0          | 0           |
| 5  | 0         | 0          | 1           |

Let's move on to some exercises.


1. Familiarize yourself with the `get_dummies` function in the `pandas` library using its documentation and other sources (e.g., web search).
2. Apply the `get_dummies` function to discretize the `REASON` feature. Assign meaningful column names to the resulting dummy variables.
3. Verify the result of the previous dummy-coding step. You should see that the `get_dummies` function has created three binary variables, one for each of the three levels of the *REASON* feature. Recalling the functioning of regression methods, you now face a problem. The three new dummy variables are **linearly dependent**. Regression models will not work with linearly dependent features. Redo task 3 but this time pay attention to the argument `drop_first`, which the `get_dummies` function offers. Set this argument in such a way that you avoid linear dependency among the created dummy variables. 

Additional, more advanced exercises:<br>

4. This task assumes you solved exercise 7 from the above tasks concerning discretization. If you have not done so, go back to the discretization exercises and solve all tasks including task 7.
5. Discretize all numerical features in the data frame using the custom function you created in task 7. Apply the `get_dummies` function to the discretized features in the resulting data frame while avoiding linear dependency. Next, create a data frame `df_all_dummy` that includes only the dummy-encoded features. 
6. Fit a logistic regression model to the `df_all_dummy` data frame. Use the class `LogisticRegression` from the module `sklearn.linear_model`. The handling of `LogisticRegression` is similar to the handling of `LinearRegression`, which we examined in [Tutorial 5](https://github.com/Humboldt-WI/IPML/blob/main/tutorial_notebooks/5_SML_for_regression_solutions.ipynb). Train the model using the `fit()` method. Then, print the estimated coefficients and compute the model's performance on the training set using the method `score()`.

In [ ]:
# 1. Familiarize yourself with the `get_dummies` function in the `pandas` library using its documentation and other sources (e.g., web search).
help(pd.get_dummies)

In [ ]:
# 2. Apply the `get_dummies` function to discretize the `REASON` feature. Assign meaningful column names to the resulting dummy variables.
reason_dummies = pd.get_dummies(df["REASON"], prefix="REASON")
reason_dummies

In [ ]:
# 3. Verify the result of the previous dummy-coding step. 
# As shown above, the `get_dummies` function has created three binary variables, one for each of the three levels of the *REASON* feature. Recalling the functioning of regression methods, you now face a problem. 
# The three new dummy variables are **linearly dependent**. Regression models will not work with linearly dependent features. Redo task 3 but this time pay attention to the argument `drop_first`, which the `get_dummies` function offers. Set this argument in such a way that you avoid linear dependency among the created dummy variables. 
reason_dummies = pd.get_dummies(df["REASON"], prefix="REASON", drop_first=True)
reason_dummies

In [33]:
# 4. This task assumes you solved exercise 7 from the above tasks concerning discretization. If you have not done so, go back to the discretization exercises and solve all tasks including task 7.

def discretize_features(df, features, bins=5, labels=None):
    """
    Discretizes the specified columns of a DataFrame into equal-width bins.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data to be discretized.
    columns (list of str): The list of column names to be discretized.
    bins (int): The number of equal-width bins to use for discretization.

    Returns:
    pandas.DataFrame: A new DataFrame with the specified columns discretized into bins.

    """
    df_discretized = df.copy()
    for feature in features:
        df_discretized[feature + "_bins"] = pd.cut(df_discretized[feature], bins=bins, labels=labels)
    return df_discretized


In [ ]:

# 5. Discretize all numerical features in the data frame using the custom function you created in task 7. 
ix_numerical = df.select_dtypes(include=np.number).columns
df_discretized_all = discretize_features(df, ix_numerical, bins=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Create a version of the data frame with only the discretized features
df_only_discrete = df_discretized_all.select_dtypes(exclude=np.number)
df_only_discrete.drop(columns=['REASON', 'JOB'], inplace=True)  # we remove the target variable

# Apply the `get_dummies` function to the discretized features in the resulting data frame while avoiding linear dependency. Next, create a data frame `df_all_dummy` that includes only the dummy-encoded features.
df_all_dummy = pd.get_dummies(df_only_discrete, drop_first=True)    # drop_first=True to avoid linear dependency
df_all_dummy


In [ ]:
# 6  Fit a logistic regression model to the `df_all_dummy` data frame and print the estimated coefficients. 
from sklearn.linear_model import LogisticRegression

# Split data into features and target
X = df_all_dummy
y = df_all_dummy.pop("BAD")

lr = LogisticRegression()
lr.fit(X, y)

print('Estimated logistic regression coefficients:\n', lr.coef_)
print(f'Model score is {lr.score(X, y):.3f}')
